In [ ]:
!pip install psycopg2 pandas

# **IMPORTS**

In [1]:
import psycopg2
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from datetime import datetime

spacy.prefer_gpu()

import os
from tqdm import tqdm
from spacy.tokens import DocBin

import matplotlib.pyplot as plt
import numpy as np

import locale
locale.getpreferredencoding = lambda: "UTF-8"

from google.colab import drive

import spacy
from spacy import displacy


In [2]:
# Mount Google Drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/AndElementTask/And Elements/'

Mounted at /content/drive


# **FETCHING DATA FOR DB**

In [11]:
# PostgreSQL database connection details
db_params = {
    "host": "viaduct.proxy.rlwy.net",
    "port": 52900,
    "database": "railway",
    "user": "postgres",
    "password": "D*eeE12*6gE*BF-G*E33cccfEb*2CB*c"
}

# SQL query to fetch data
sql_query_keywords = """
select
    c.id, cat.category_name as keyword, cat.category_type as category
from
    public.categories cat
inner join
    public.category_article_relationship car
    on cat.id = car.category_id
inner join
    public."content" c
    on c.id = car.article_id
where c.is_reviewed = true
order by c.id

"""

sql_query_articles = """
select
    c.id, c.title || ' ' || c."content" || ' ' || c.author AS content
from
    public."content" c
where c.is_reviewed = true
order by c.id
"""

# Output CSV file path
keywords_csv_path = "keywords.csv"
articles_csv_path = "articles.csv"

try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_params)

    # Use pandas to fetch the data
    keywords = pd.read_sql_query(sql_query_keywords, connection)
    articles = pd.read_sql_query(sql_query_articles, connection)

    # Export the DataFrame to a CSV file
    keywords.to_csv(keywords_csv_path, index=False,encoding="utf-8")
    articles.to_csv(articles_csv_path, index=False,encoding="utf-8")

    print(f"Data successfully exported!")

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the database connection
    if connection:
        connection.close()


<ipython-input-11-b0678ea13553>:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  keywords = pd.read_sql_query(sql_query_keywords, connection)
<ipython-input-11-b0678ea13553>:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  articles = pd.read_sql_query(sql_query_articles, connection)


Data successfully exported!


In [12]:
# Load CSV files
keywords_df = pd.read_csv("keywords.csv",encoding="utf-8")
articles_df = pd.read_csv("articles.csv",encoding="utf-8")

# Merge the two datasets on the "id" column
merged_df = pd.merge(keywords_df, articles_df, on="id")

In [13]:
# Drop rows with any NaN (null) values
merged_df = merged_df.dropna()

# Filter rows
merged_df = merged_df[merged_df['keyword'].apply(lambda x: len(str(x).split()) <= 3)]

# Output the filtered dataframe
print(merged_df)

           id               keyword       category  \
0           1           feb 18 2024          Crime   
1           1           jan 12 2024          Crime   
2           1                 hosts           Tags   
3           1                voting           Tags   
4           1                  fans           Tags   
...       ...                   ...            ...   
367704  21365               zhoresh     Characters   
367705  21365  book recommendations  Type_of_Story   
367706  21365      nuclear disaster           Tags   
367707  21365  andrew leatherbarrow     Characters   
367709  21365  absolutely essential       Sentence   

                                                  content  
0       The 2024 People's Choice Awards Host and Nomin...  
1       The 2024 People's Choice Awards Host and Nomin...  
2       The 2024 People's Choice Awards Host and Nomin...  
3       The 2024 People's Choice Awards Host and Nomin...  
4       The 2024 People's Choice Awards Host and No

In [14]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 303163 entries, 0 to 367709
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        303163 non-null  int64 
 1   keyword   303163 non-null  object
 2   category  303163 non-null  object
 3   content   303163 non-null  object
dtypes: int64(1), object(3)
memory usage: 11.6+ MB


# **HELPER FUNCTIONS**

In [15]:
def preprocess_text(text):
    """
    Preprocess the text by:
    - Replacing periods in emails and URLs with spaces.
    - Replacing special characters (!, +, -, @, em dashes, quotation marks) with spaces.
    - Handling URLs (e.g., http://example becomes http example).
    - Removing special characters between words.
    - Removing trailing spaces and converting text to lowercase.
    - Removing all stop words.

    Args:
    - text (str): The input text to preprocess.

    Returns:
    - str: The cleaned text
    """
    text = str(text)  # Ensure the input is a string

    # Handle URLs by replacing ":" and "/" with spaces
    text = re.sub(r"http[s]?://", "http ", text)  # Replace http:// or https:// with "http"
    text = re.sub(r"[/:]", " ", text)  # Replace remaining ":" and "/" with spaces

    # Remove periods in emails and URLs
    text = re.sub(r"(?<=\S)\.(?=\S)", " ", text)

    # Replace specific special characters (!, +, -, @, em dash, quotation marks, etc.) with spaces
    text = re.sub(r"[!+\-@“”\"‘’—–]", " ", text)

    # Replace special characters between words
    text = re.sub(r"(?<=\w)[^\w\s,](?=\w)", " ", text)

    # Convert to lowercase
    text = text.lower()

    # Remove extra spaces and trim leading/trailing spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

def split_text_into_chunks(text, chunk_size=500, overlap=50):
    """
    Split the text into chunks of specified size with overlap.

    Args:
    - text (str): The input text.

    Returns:
    - str: Chunks
    """
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
        if i + chunk_size >= len(words):
            break
    return chunks

In [16]:
def extract_sentences_with_keywords(article, keywords):
    """
    Extract sentences containing any of the given keywords and their surrounding sentences (±1 sentence).

    Args:
    - article (str): The full article text.
    - keywords (list): List of keywords to look for.

    Returns:
    - str: Filtered article with relevant sentences and their context.
    """
    # Split the article into sentences using regex for sentence boundaries
    sentences = re.split(r'(?<=[.!?])\s+', article)

    # Find sentences with any of the keywords
    keyword_indices = [
        idx for idx, sentence in enumerate(sentences)
        if any(re.search(rf"\b{re.escape(keyword)}\b", sentence, flags=re.IGNORECASE) for keyword in keywords)
    ]

    # Include surrounding sentences (±1 sentence for context)
    relevant_indices = set()
    for idx in keyword_indices:
        relevant_indices.update([idx - 1, idx, idx + 1])  # Include previous, current, and next sentences

    # Ensure indices are within valid range
    relevant_indices = sorted(idx for idx in relevant_indices if 0 <= idx < len(sentences))

    # Extract relevant sentences
    filtered_sentences = [sentences[idx] for idx in relevant_indices]

    return " ".join(filtered_sentences)

def create_annotations_with_context_and_chunks(merged_df, chunk_size=512, overlap=50):
    """
    Create training annotations using relevant sentences with keywords and their surrounding context,
    while breaking the filtered text into manageable chunks.

    Args:
    - merged_df (pd.DataFrame): DataFrame containing 'id', 'cleaned_content', 'cleaned_keyword', and 'category'.
    - chunk_size (int): Maximum size (in characters) for each chunk of filtered text.

    Returns:
    - tuple: (training_data, label_missing_counts, label_correct_counts)
    """
    training_data = []
    label_missing_counts = {label: 0 for label in merged_df["category"].unique()}  # Initialize counts for labels
    label_correct_counts = {label: 0 for label in merged_df["category"].unique()}

    for article_id, group in merged_df.groupby("id"):
        try:
            full_text = group["cleaned_content"].iloc[0]  # Full article text
            keywords = group["cleaned_keyword"].tolist()  # List of keywords for the article
            categories = group["category"].tolist()  # Corresponding categories for the keywords

            # Extract relevant sentences using keywords
            filtered_text = extract_sentences_with_keywords(full_text, keywords)

            # Split the filtered text into chunks
            chunks = split_text_into_chunks(filtered_text, chunk_size, overlap)

            for chunk in chunks:
                entities = []
                has_keyword = False

                for keyword, label in zip(keywords, categories):
                    # Check if the keyword exists in the current chunk
                    if re.search(r'\b' + re.escape(keyword) + r'\b', chunk, flags=re.IGNORECASE):
                        has_keyword = True
                        for match in re.finditer(r'\b' + re.escape(keyword) + r'\b', chunk, flags=re.IGNORECASE):
                            start = match.start()
                            end = match.end()
                            label_correct_counts[label] += 1
                            entities.append((start, end, label))
                    else:
                        label_missing_counts[label] += 1

                if has_keyword:
                    training_data.append((chunk, {"entities": entities}))

        except Exception as e:
            print(f"Error processing article ID {article_id}: {e}")

    return training_data, label_missing_counts, label_correct_counts

def create_annotations_with_chunks_single_entity(merged_df, chunk_size=500, overlap=50):
    training_data = []
    label_missing_counts = {label: 0 for label in merged_df["category"].unique()}  # Initialize counts for labels
    label_correct_counts = {label: 0 for label in merged_df["category"].unique()}

    for article_id, group in merged_df.groupby("id"):
        try:
            text = group["cleaned_content"].iloc[0]
            chunks = split_text_into_chunks(text, chunk_size, overlap)  # Split text into chunks

            for chunk in chunks:
                has_keyword = False
                entities = []
                for _, row in group.iterrows():
                    keyword = row["cleaned_keyword"]
                    label = row["category"]

                    # Check if the keyword exists in the chunk
                    if re.search(r'\b' + re.escape(keyword) + r'\b', chunk, flags=re.IGNORECASE):
                        has_keyword = True
                        for match in re.finditer(r'\b' + re.escape(keyword) + r'\b', chunk, flags=re.IGNORECASE):
                            start = match.start()
                            end = match.end()
                            entities.append((start, end, label))
                            break

                if has_keyword:
                  label_correct_counts[label] += 1
                  training_data.append((chunk, {"entities": entities}))
                else:
                  label_missing_counts[label] += 1
                  training_data.append((chunk, {"entities": []}))

        except Exception as e:
            print(f"Error processing article ID {article_id}: {e}")

    return training_data, label_missing_counts, label_correct_counts

# Main function to create annotations
def create_annotations_with_chunks(merged_df, chunk_size=500, overlap=50):
    training_data = []
    label_missing_counts = {label: 0 for label in merged_df["category"].unique()}  # Initialize counts for labels
    label_correct_counts = {label: 0 for label in merged_df["category"].unique()}

    for article_id, group in merged_df.groupby("id"):
        try:
            text = group["cleaned_content"].iloc[0]
            chunks = split_text_into_chunks(text, chunk_size, overlap)  # Split text into chunks

            for chunk in chunks:
                has_keyword = False
                entities = []
                for _, row in group.iterrows():
                    keyword = row["cleaned_keyword"]
                    label = row["category"]

                    # Check if the keyword exists in the chunk
                    if re.search(r'\b' + re.escape(keyword) + r'\b', chunk, flags=re.IGNORECASE):
                        has_keyword = True
                        for match in re.finditer(r'\b' + re.escape(keyword) + r'\b', chunk, flags=re.IGNORECASE):
                            start = match.start()
                            end = match.end()
                            label_correct_counts[label] += 1
                            entities.append((start, end, label))
                            break
                    else:
                        label_missing_counts[label] += 1

                if has_keyword:
                    training_data.append((chunk, {"entities": entities}))

        except Exception as e:
            print(f"Error processing article ID {article_id}: {e}")

    return training_data, label_missing_counts, label_correct_counts

def create_annotations(merged_df):
    training_data = []
    label_missing_counts = {label: 0 for label in merged_df["category"].unique()}  # Initialize counts for labels
    label_correct_counts = {label: 0 for label in merged_df["category"].unique()}

    for article_id, group in merged_df.groupby("id"):
        try:
            text = group["cleaned_content"].iloc[0]  # Full article text
            entities = []
            has_keyword = False

            for _, row in group.iterrows():
                keyword = row["cleaned_keyword"]
                label = row["category"]

                # Check if the keyword exists in the article
                if re.search(r'\b' + re.escape(keyword) + r'\b', text, flags=re.IGNORECASE):
                    has_keyword = True
                    for match in re.finditer(r'\b' + re.escape(keyword) + r'\b', text, flags=re.IGNORECASE):
                        start = match.start()
                        end = match.end()
                        label_correct_counts[label] += 1
                        entities.append((start, end, label))
                else:
                    label_missing_counts[label] += 1

            if has_keyword:
                training_data.append((text, {"entities": entities}))

        except Exception as e:
            print(f"Error processing article ID {article_id}: {e}")

    return training_data, label_missing_counts, label_correct_counts

def save_training_data_to_csv(training_data, output_file):
    """
    Save training data to a CSV file in the format:
    article_id, article, Entities

    Args:
    - training_data (list): A list of tuples containing text and entity annotations.
    - output_file (str): The path to save the CSV file.
    """
    data_rows = []
    for article_id, (text, annotation) in enumerate(training_data):
        # Extract the entities as a list of tuples (start, end, label)
        entities = [(start, end, label) for start, end, label in annotation["entities"]]

        # Prepare a dictionary for the row
        data_rows.append({
            "article_id": article_id,
            "article": text,
            "Entities": str(entities)  # Convert list of tuples to string format
        })

    # Convert to DataFrame
    df = pd.DataFrame(data_rows)

    # Save to CSV
    df.to_csv(output_file, index=False)
    print(f"Training data saved to {output_file}")

# **DATA PREPROCESSING**

In [17]:
# Apply preprocessing to the 'content' column
merged_df['cleaned_content'] = merged_df['content'].apply(preprocess_text)
# Apply preprocessing to the 'keyword' column
merged_df['cleaned_keyword'] = merged_df['keyword'].apply(preprocess_text)

In [18]:
filtered_df = merged_df[merged_df['id'] < 2000]
len(filtered_df)

48364

# **FILTERING DATA**

In [ ]:
tags_df = merged_df[merged_df['category'] == 'Tags']

# Count the frequency of each keyword in the cleaned_keyword column
keyword_counts = tags_df['keyword'].value_counts()
# Convert to a DataFrame for easier handling
keyword_counts_df = keyword_counts.reset_index()
keyword_counts_df.columns = ['keyword', 'count']
keyword_counts_df

In [ ]:
setting_df = merged_df[merged_df['category'] == 'Setting']

# Count the frequency of each keyword in the cleaned_keyword column
keyword_counts = setting_df['keyword'].value_counts()
# Convert to a DataFrame for easier handling
keyword_counts_df_2 = keyword_counts.reset_index()
keyword_counts_df_2.columns = ['keyword', 'count']
keyword_counts_df_2

In [ ]:
# Filter keywords
filtered_tags_keywords = keyword_counts_df[keyword_counts_df['count'] > 100]['keyword']
filtered_setting_keywords = keyword_counts_df_2[keyword_counts_df_2['count'] > 20]['keyword']

# Filter tags_df to keep only rows with the filtered keywords
filtered_df = merged_df[(merged_df['keyword'].isin(filtered_setting_keywords) & (merged_df['category'] == 'Setting')) | (merged_df['keyword'].isin(filtered_tags_keywords) & (merged_df['category'] == 'Tags'))]

# Display the filtered dataframe
print(filtered_df)

# **CREATE ANNOTATIONS WITH CHUNKS**

In [19]:
training_data, label_missing_counts, label_correct_counts = create_annotations_with_context_and_chunks(filtered_df)

In [ ]:
training_data, label_missing_counts, label_correct_counts = create_annotations_with_chunks_single_entity(filtered_df)

In [ ]:
training_data, label_missing_counts, label_correct_counts = create_annotations_with_chunks(filtered_df)

In [ ]:
training_data, label_missing_counts, label_correct_counts = create_annotations(filtered_df)

Plotting Annotations

In [ ]:
# Combine all unique labels from both dictionaries
all_labels = set(label_correct_counts.keys()).union(label_missing_counts.keys())

# Ensure all labels have counts in both dictionaries (default to 0 if missing)
correct_counts = [label_correct_counts.get(label, 0) for label in all_labels]
missing_counts = [label_missing_counts.get(label, 0) for label in all_labels]

# Bar plot configuration
x = np.arange(len(all_labels))  # Label locations
width = 0.5  # Bar width

# Create the plot
fig, ax = plt.subplots(figsize=(15, 8))
bars_correct = ax.bar(x - width / 2, correct_counts, width, label='Correct Counts', color='green')
bars_missing = ax.bar(x + width / 2, missing_counts, width, label='Missing Counts', color='red')

# Add labels, title, and legend
ax.set_xlabel('Labels', fontsize=12)
ax.set_ylabel('Counts', fontsize=12)
ax.set_title('Comparison of Correct and Missing Counts by Label', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(all_labels, rotation=45, ha="right", fontsize=12)
ax.legend()

# Display counts on top of bars
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 2),  # Offset text above bar
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(bars_correct)
add_labels(bars_missing)

# Show the plot
plt.tight_layout()
plt.show()

# **CHECKING SPANS**

In [22]:
# Removing overlapping spans
for text, annotations in training_data:
    spans = annotations["entities"]
    # Sort the spans by their start position
    spans = sorted(spans, key=lambda span: span[0])

    # List to hold non-overlapping spans
    non_overlapping_spans = []

    for span in spans:
        # If non_overlapping_spans is empty, add the first span directly
        if not non_overlapping_spans:
            non_overlapping_spans.append(span)
        else:
            # Get the last span in the non_overlapping_spans list
            last_span = non_overlapping_spans[-1]

            # Check for overlap with the last span
            if last_span[1] > span[0]:
                # If overlapping, keep the span with the greater length
                if (last_span[1] - last_span[0]) < (span[1] - span[0]):
                    non_overlapping_spans[-1] = span  # Replace with the longer span
            else:
                # If no overlap, add the span to the list
                non_overlapping_spans.append(span)

    # Replace the original spans with the filtered non-overlapping spans
    annotations["entities"] = non_overlapping_spans

In [23]:
for text, annotations in training_data:
    spans = annotations["entities"]
    spans = sorted(spans, key=lambda span: span[0])
    for i in range(len(spans)):
        for j in range(i + 1, len(spans)):
            if spans[i][1] > spans[j][0]:  # Check for overlap
                print(f"Overlapping spans found - {i}: {spans[i]} and {spans[j]} in text: {text}")

In [24]:
for text, annotations in training_data:
    spans = annotations["entities"]
    for span in spans:
        start, end, label = span
        span_text = text[start:end]
        # print(f"Span: {span}, Text: '{span_text}'")

        # Check for leading/trailing whitespace
        if span_text != span_text.strip():
            print(f"Warning: Span '{span_text}' contains leading/trailing whitespace.")

        # Check if the span is within valid bounds
        if start < 0 or end > len(text):
            print(f"Error: Span {span} is out of text bounds.")

        # Check for duplicate spans
        if spans.count(span) > 1:
            print(f"Duplicate span found: {span}")

In [49]:
for i in range(len(training_data)):
    text = training_data[i][0]
    annotations = training_data[i][1]
    for span in annotations["entities"]:
        start, end, label = span
        if text[start-1] in ".,!?()[]{}'\"" or text[end:end+1] in ".,!?()[]{}'\"":
            print(f"Span adjacent to punctuation {i}: {span}, Text: '{text[start:end]}'")



Span adjacent to punctuation 102: (2953, 2958, 'Tags'), Text: 'actor'
Span adjacent to punctuation 676: (2923, 2931, 'Tags'), Text: 'homicide'
Span adjacent to punctuation 847: (3031, 3044, 'Tags'), Text: 'mental health'


Display Spans

In [26]:
# Load a blank English model for custom annotations
nlp = spacy.blank("en")
sample = training_data[6]
print(sample[1])
print(sample[0])
# Sample text with the specified entity annotation
text = sample[0]
entities = sample[1]["entities"]
# Create a Doc object
doc = nlp(text)

# Manually set the entity with the specified span and label
doc.ents = [nlp.make_doc(text).char_span(ent[0], ent[1], label=ent[2]) for ent in entities]

# Visualize the text using displacy
displacy.render(doc, style="ent", jupyter=True)

{'entities': [(24, 40, 'Killers'), (60, 73, 'Tags'), (75, 91, 'Killers'), (94, 107, 'Tags'), (150, 163, 'Tags'), (194, 203, 'Location'), (391, 403, 'Victim'), (433, 453, 'Tags'), (535, 544, 'Setting'), (633, 643, 'Tags'), (645, 661, 'Tags'), (891, 904, 'Tags'), (1107, 1123, 'Killers'), (1182, 1192, 'Tags'), (1723, 1729, 'Setting'), (1979, 1992, 'Location'), (1994, 2003, 'Location'), (2356, 2371, 'Characters'), (2408, 2421, 'Location'), (2574, 2578, 'Crime'), (2583, 2587, 'Crime'), (2659, 2663, 'Crime'), (2781, 2787, 'Setting')]}
is '90 day fiance' star geoffrey paschel revealing his full criminal past? geoffrey paschel s criminal past has been a central theme of this season s 90 day fiancé before the 90 days but is the tennessee resident divulging his entire criminal history to his russian love interest? paschel confessed on the show to serving time in prison more than twenty years ago for dealing drugs, telling varya malina, the woman he met through an international dating site, that 

Checking for Valid and Invalid spans

In [ ]:
nlp = spacy.load("en_core_web_sm")
db = DocBin()  # Create a DocBin object

for i in range(len(training_data)):  # Loop through the training data
  text, annot = training_data[i]
  doc = nlp.make_doc(text)  # Create a spaCy Doc object from the text

  # Validate and add spans to the Doc
  valid_spans = []
  for start, end, label in annot['entities']:
      if 0 <= start < len(text) and 0 < end <= len(text):  # Ensure indices are within bounds
          span = doc.char_span(start, end, label=label, alignment_mode="strict")
          if span:  # Ensure the span is valid
              valid_spans.append(span)
          else:
              print(f"Skipping invalid span: {start}-{end} for label {i} {label}")

      else:
          print(f"Skipping out-of-range span: {start}-{end} for label {label}")


In [ ]:
db = DocBin()  # Create a DocBin object

for i in range(19040 ,19041):  # Loop through the training data
  text, annot = training_data[i]
  doc = nlp.make_doc(text)  # Create a spaCy Doc object from the text
  for token in doc:
    print(f"Token: '{token.text}' | Start: {token.idx} | End: {token.idx + len(token)}")

  # Validate and add spans to the Doc
  valid_spans = []
  for start, end, label in annot['entities']:
      if 0 <= start < len(text) and 0 < end <= len(text):  # Ensure indices are within bounds
          span = doc.char_span(start, end, label=label, alignment_mode="contract")
          if span:  # Ensure the span is valid
              valid_spans.append(span)
              # print(f"Valid span: {span} for label {label}")
          else:
              print(f"Skipping invalid span: {start}-{end} for label {i} {label}")

      else:
          print(f"Skipping out-of-range span: {start}-{end} for label {label}")

# **SAVING ANNOTATIONS**

In [61]:
# Save the training data
save_training_data_to_csv(training_data, drive_path + "preprocessed_data.csv")

Training data saved to /content/drive/MyDrive/AndElementTask/And Elements/preprocessed_data_tags.csv
